In [41]:
from random import random
import numpy as np
from simple_wta import WTAProblem

def dist(prob: WTAProblem, x, y):
    return np.sum(prob.v*(x-y)**2)

# I made this too fucking complicated, in reality things are quite simple

def agent_update(prob: WTAProblem, labels: list, i: int, k: int):
    (n,m) = prob.p.shape
    S = [[j for j in range(n) if labels[j]==c] for c in range(k)]
    means = [np.average(prob.p[S[c],:],axis=0) for c in range(k)]
    x = prob.p[i,:]
    label = np.argmin([dist(prob,x,y) for y in means])
    return label

# k-means is a dumb as fuck algorithm
def init(prob: WTAProblem, k: int):
    (n,m) = prob.p.shape
    # choose one center uniformly at random over the data
    lb = np.min(prob.p,axis=0)
    ub = np.max(prob.p,axis=0)
    centroids = [np.random.uniform(low=lb,high=ub)]
    # Randomly select weapons as new centroids until k centroids are chosen
    points = list(range(n))
    for i in range(1,k):
        D = []
        for j in points:
            x = prob.p[j,:]
            D.append(np.min([dist(prob,x,y) for y in centroids])) # this can be optimized to remove redundant distance computations
        idx = points[np.random.choice(len(points),p=D/np.sum(D))]
        new_centroid = prob.p[idx,:]
        centroids.append(new_centroid)
        points.remove(idx)
    # assign each point to its closest centroid
    labels = [np.argmin([dist(prob,prob.p[i,:],centroids[j]) for j in range(k)]) for i in range(n)]
    return centroids, labels

""" A naive kmeans algorithim initialized with kmeans++"""
def distributed_kmeans(prob: WTAProblem, k:int, maxiters=100):
    (n,m) = prob.p.shape
    centroids, old_labels = init(prob, k)
    iters = 0
    while iters < maxiters:
        print(iters)
        iters+=1
        labels = [agent_update(prob,old_labels,i,k) for i in range(n)]
        if old_labels == labels:
            break
        else:
            old_labels = labels
    return iters, labels

0.012688580000000229